In [ ]:
access_token = ""

In [2]:
idCuda = 3
idTorch = 0

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = f"{str(idCuda)}"

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import torch

In [5]:
torch.cuda.device_count()

1

In [6]:
torch.cuda.empty_cache()
torch.cuda.set_device(idTorch)

if torch.cuda.is_available():
    GPU = torch.cuda.get_device_name(idTorch)
    print(f"PyTorch available GPU: {GPU}")
else:
    print("GPU not available")

PyTorch available GPU: NVIDIA RTX A5000


In [7]:
device = torch.device(f"cuda:{idTorch}" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

Device: cuda:0


In [ ]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=5, token=access_token)

# Prepare your dataset (replace with your own dataset loading and preprocessing)
# Example code assumes you have lists `texts` and `labels` containing your data
texts = [...]  # List of text samples
labels = [...]  # List of corresponding labels (0 to 4 for 5 classes)

# Tokenize texts and convert labels to tensors
max_length = 128
tokenized_texts = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
labels = torch.tensor(labels)

# Split dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(tokenized_texts, labels, test_size=0.2, random_state=42)

# Define training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=100,
    evaluation_strategy='steps',
    save_strategy='steps',
    save_steps=500,
    eval_steps=500,
    output_dir='./results',
    overwrite_output_dir=True,
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_texts,
    eval_dataset=val_texts,
    compute_metrics=None,  # Add your metrics function here if needed
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()
